In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import numpy as np
import  pandas as pd

from sklearn.naive_bayes import GaussianNB,ComplementNB,MultinomialNB
from sklearn.mixture import BayesianGaussianMixture

from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix as cm
from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.neighbors.kde import KernelDensity

from IPython.display import display_html

from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
import statsmodels as sm
import matplotlib

from IPython.display import display,clear_output,HTML

In [2]:
#### Source= https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

def mydisplay(dfs, names=[]):
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

In [3]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Infected/Unifected Users')
display(train.Target.value_counts())
user_Dec=train.index.values

Infected/Unifected Users


1    2910
0    2910
Name: Target, dtype: int64

In [4]:
jan=pd.read_csv('Feature-Set_Jan_Final.csv',index_col='User')
print('Infected/Unifected Users')
display(jan.Target.value_counts())
user_jan=jan.index.values

Infected/Unifected Users


0    6406
1     532
Name: Target, dtype: int64

In [5]:
no=[user for user in user_jan if user not in user_Dec]

o=[user for user in user_jan if user in user_Dec]

jan.drop(o,inplace=True)

print("After eliminating overlapping users:")
print(jan.Target.value_counts())

After eliminating overlapping users:
0    6181
1     532
Name: Target, dtype: int64


In [6]:
z=jan.sample(frac=1,random_state=10)
display(z.Target.value_counts())

0    6181
1     532
Name: Target, dtype: int64

## Individual

### Parametric approach

In [7]:
Y_test=z.Target.values

for col in train.columns[:-1]:
    train[col]=np.log(train[col])
train.replace([np.inf,-np.inf],0,inplace=True)

for col in z.columns[:-1]:
    z[col]=np.log(z[col])
z.replace([np.inf,-np.inf],0,inplace=True)

In [38]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
# scaler=StandardScaler()
# scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
clf = MultinomialNB()
clf.fit(X_train.reshape(-1,1),Y_train)
X_test=z[col].values.reshape(-1,1)
non_mal_lst=clf.predict_proba(X_test)

In [8]:
lst=[]
for col in train.columns[:-1]:
    
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = ComplementNB()
    clf.fit(scaled_X,Y_train)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    mean=clf.theta_[0][0]
    std=np.sqrt(clf.sigma_[0][0])
    non_mal_lst=[]
    for x in X_test:
        non_mal_lst.append((norm.pdf(x[0],mean,std)))
    
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = ComplementNB()
    clf.fit(scaled_X,Y_train)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    mean=clf.theta_[0][0]
    std=np.sqrt(clf.sigma_[0][0])
    mal_lst=[]
    for x in X_test:
        mal_lst.append((norm.pdf(x[0],mean,std)))
    
    
    df=pd.DataFrame({'Non_Mal':non_mal_lst,'Mal':mal_lst,
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)


    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    df['Predicted']=np.where(df['Non_Mal'] <= df['Mal'], 'Malicious','Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Predicted,df.Ground_Truth))
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Parametric/Jan/Para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,249,583
Non_Malicious,283,5598


-----------------------------------------------------------------------------------------------------
DP



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,177,605
Non_Malicious,355,5576


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,339,1273
Non_Malicious,193,4908


-----------------------------------------------------------------------------------------------------
2300



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,337,1302
Non_Malicious,195,4879


-----------------------------------------------------------------------------------------------------


## Combined

In [8]:
col=train.columns[:-1]

In [66]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=100)
clf.fit(scaled_X,Y_train)

X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

In [67]:
m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

In [68]:
m.head()

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,3.168548e-01,6.831452e-01,Non_Malicious,Non_Malicious
1,1.000000e+00,1.877022e-27,Non_Malicious,Malicious
2,1.173480e-08,1.000000e+00,Non_Malicious,Non_Malicious
3,4.121739e-09,1.000000e+00,Non_Malicious,Non_Malicious
4,1.000000e+00,4.588993e-53,Non_Malicious,Malicious


In [69]:
pd.crosstab(m.Ground_Truth,m.Predicted)

Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,434,98
Non_Malicious,5939,242


In [45]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train.reshape(-1,1))    
clf = BayesianGaussianMixture()
clf.fit(scaled_X,Y_train)
X_test=scaler.transform(z[col].values.reshape(-1,1))
non_mal_lst=clf.predict_proba(X_test)

In [46]:
non_mal_lst

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [11]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = MultinomialNB()
clf.fit(scaled_X,Y_train)
X_test=scaler.transform(z[col].values)
non_mal_lst=clf.predict_proba(X_test)

ValueError: Input X must be non-negative

In [18]:
clf.theta_

array([[  4.10477303e-16,   4.42944650e-16,   1.19797261e-17,
          1.79810348e-16,   1.59608867e-16,   1.08695031e-16,
          7.03332009e-17,  -1.61230327e-16,   8.07525104e-16,
         -8.67371276e-16,  -1.34352247e-16,  -7.45489962e-17,
         -4.78426004e-17,  -5.08947600e-17,  -7.10390128e-17,
         -1.55965351e-16,  -9.59141128e-17,   1.97627329e-17,
         -1.24909628e-16,   1.41582050e-16,   3.47183145e-17,
          1.40323034e-16,   1.66533454e-16,   6.39808939e-17,
          4.81478164e-17,  -3.90237670e-16,   3.73469869e-16,
         -1.43737637e-16,   2.65957550e-16,   5.70372310e-18,
          3.52524424e-17,  -1.23993980e-17,  -1.54477424e-16,
         -1.63519446e-16,  -1.76567428e-16]])

In [ ]:
mean=clf.theta_[0][0]
std=np.sqrt(clf.sigma_[0][0])
non_mal_lst=[]

In [9]:
for x in X_test:
        non_mal_lst.append((norm.pdf(x[0],mean,std)))

SyntaxError: invalid syntax (<ipython-input-9-f04fa38cd1f0>, line 4)